In [160]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score,f1_score,mean_absolute_error,mean_squared_error
from sklearn.impute import SimpleImputer
import xgboost
from sklearn import svm
from xgboost import XGBClassifier, plot_importance
import pickle as pk
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier


In [161]:
diabetesdataurl='diabetes.csv'

diabetesdata= pd.read_csv(diabetesdataurl)

In [162]:
diabetesdata[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = diabetesdata[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)
# plt.figure(figsize=(10,10))
# sns.heatmap(diabetesdata.corr(), annot=True)
# plt.show()

In [163]:
diabetesdata.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [164]:
# diabetesdata['Glucose'].fillna(diabetesdata['Glucose'].mean(), inplace = True)
# diabetesdata['BloodPressure'].fillna(diabetesdata['BloodPressure'].mean(), inplace = True)
# diabetesdata['SkinThickness'].fillna(diabetesdata['SkinThickness'].median(), inplace = True)
# diabetesdata['Insulin'].fillna(diabetesdata['Insulin'].median(), inplace = True)
# diabetesdata['BMI'].fillna(diabetesdata['BMI'].median(), inplace = True)
diabetesdata.dropna(inplace=True)


In [176]:
X=diabetesdata.drop(['Outcome','BloodPressure'],axis=1)
X

,Pregnancies,Glucose,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
3,1,89.0,23.0,94.0,28.1,0.167,21
4,0,137.0,35.0,168.0,43.1,2.288,33
6,3,78.0,32.0,88.0,31.0,0.248,26
8,2,197.0,45.0,543.0,30.5,0.158,53
13,1,189.0,23.0,846.0,30.1,0.398,59
...,...,...,...,...,...,...,...
753,0,181.0,44.0,510.0,43.3,0.222,26
755,1,128.0,39.0,110.0,36.5,1.057,37
760,2,88.0,26.0,16.0,28.4,0.766,22
763,10,101.0,48.0,180.0,32.9,0.171,63


In [177]:
# X.replace(0,X.mean(), inplace=True)

In [178]:
X

,Pregnancies,Glucose,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
3,1,89.0,23.0,94.0,28.1,0.167,21
4,0,137.0,35.0,168.0,43.1,2.288,33
6,3,78.0,32.0,88.0,31.0,0.248,26
8,2,197.0,45.0,543.0,30.5,0.158,53
13,1,189.0,23.0,846.0,30.1,0.398,59
...,...,...,...,...,...,...,...
753,0,181.0,44.0,510.0,43.3,0.222,26
755,1,128.0,39.0,110.0,36.5,1.057,37
760,2,88.0,26.0,16.0,28.4,0.766,22
763,10,101.0,48.0,180.0,32.9,0.171,63


In [179]:
Y=diabetesdata['Outcome']
# X=np.array(X)
X

,Pregnancies,Glucose,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
3,1,89.0,23.0,94.0,28.1,0.167,21
4,0,137.0,35.0,168.0,43.1,2.288,33
6,3,78.0,32.0,88.0,31.0,0.248,26
8,2,197.0,45.0,543.0,30.5,0.158,53
13,1,189.0,23.0,846.0,30.1,0.398,59
...,...,...,...,...,...,...,...
753,0,181.0,44.0,510.0,43.3,0.222,26
755,1,128.0,39.0,110.0,36.5,1.057,37
760,2,88.0,26.0,16.0,28.4,0.766,22
763,10,101.0,48.0,180.0,32.9,0.171,63


In [180]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


In [197]:
# xgb_cls=xgboost.XGBClassifier(learning_rate =0.0001,
#  n_estimators=1000,
 
#  max_depth=5,
#  min_child_weight=1,
#  gamma=0,
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  nthread=4,
#  scale_pos_weight=1,
#  seed=27)
classifier=LogisticRegression()
# classifier=RandomForestClassifier(n_estimators=2,criterion="entropy")
# classifier=svm.SVC(kernel='linear')
# classifier=MLPClassifier(hidden_layer_sizes=(64,64))
classifier.fit(X_train,Y_train)
pred=classifier.predict(X_train)
pred2=classifier.predict(X_test)

C:\Users\VICKFURY\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [198]:
trainaccuracy=accuracy_score(Y_train,pred)
f1score= f1_score(Y_test,pred2, average=None)
trainaccuracy, f1score

(0.7891373801916933, array([0.87850467, 0.74509804]))

In [199]:
# (pd.Series(classifier.feature_importances_, index=X.columns)
# # .nlargest(6)
#    .plot(kind='barh'))


In [200]:
preds=classifier.predict(X_test)
preds

array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int64)

In [201]:
Y_test

715    1
700    0
325    0
648    1
128    1
      ..
169    0
617    0
63     0
395    0
709    1
Name: Outcome, Length: 79, dtype: int64

In [202]:
testaccuracy=accuracy_score(Y_test,preds)*100
testaccuracy

83.54430379746836

In [187]:
f1score=f1_score(Y_test,preds)
mae=mean_squared_error(Y_test,preds)
print(f1score,mae)

0.7755102040816326 0.13924050632911392


In [188]:
scaler = StandardScaler()
sample1=[[1,89,66,23,94,28.1,21]]
# sample1=[5,116,74,0,0,25.6,30]
# sample1=[[6,148,72,35,0,33.6,50]]
pregnanciesno=1
glucose=89
bmi=28.1
insulin=94
Age=21
bloodpressure=66
SkinThickness=23
features=[pregnanciesno,glucose,bloodpressure,SkinThickness,insulin,bmi,Age]
features=np.array(features)
features=features.reshape(1,-1)
predictionvalue=classifier.predict(features)

print(predictionvalue)
if predictionvalue==1:
    print ('You have diabetes')
else:
    print ('No diabetes')

[1]
You have diabetes


C:\Users\VICKFURY\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [189]:
ml= "diabetespredmodelusingxgboost.pkl"
# ml= "diabetespredmodelusingxgboost.json"
md=pk.dump(classifier,open(ml,'wb'))
# import joblib as jb
# classifier.save_model(ml)
# jb.dump(classifier,ml)


In [190]:
model =pk.load(open(ml,'rb'))
print(model)
# xgb_cls.load_model(ml)
# model=jb.load(ml)
sample1=[5,116,74,0,0,25.6,30]
sample1=[6,148,72,35,0,33.6,50]
sample1=np.array(sample1)
sample1=sample1.reshape(-1,1)
sample1=scaler.fit_transform(sample1)
sample1=sample1.reshape(1,-1)
print(sample1)
model.predict(sample1)


SVC(kernel='linear')
[[-0.93341478  2.1327263   0.49169305 -0.30723104 -1.06297004 -0.3374606
   0.0166571 ]]


C:\Users\VICKFURY\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([0], dtype=int64)

In [381]:
xgboost.__version__

'1.5.0'